In [2]:
import pandas as pd 
import os 
import requests
from bs4 import BeautifulSoup
import requests
import nltk
import openpyxl
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords


In [4]:
os.getcwd()

'C:\\Users\\Dell\\Internship\\Blackcoffer'

In [3]:
os.chdir("E:\Internship\Blackcoffer")

In [3]:
Artical=pd.read_excel("Input.xlsx")

In [4]:
Artical.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [8]:
Artical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   URL_ID  114 non-null    float64
 1   URL     114 non-null    object 
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


In [1]:


url = "https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    article_title = soup.find('h1', class_='entry-title').text.strip()
    
    article_text = ""
    paragraphs = soup.find_all('p')
    for paragraph in paragraphs:
        article_text += paragraph.text + "\n"
    
    print("Article Title:", article_title)
    print("Article Text:\n", article_text)
else:
    print("Failed to retrieve the webpage.")



Article Title: Rise of telemedicine and its Impact on Livelihood by 2040
Article Text:
 Automate the Data Management Process
Realtime Kibana Dashboard for a financial tech firm
Data Management, ETL, and Data Automation
Data Management – EGEAS
How To Secure (SSL) Nginx with Let’s Encrypt on Ubuntu (Cloud VM, GCP, AWS, Azure, Linode) and Add Domain
Deploy and view React app(Nextjs) on cloud VM such as GCP, AWS, Azure, Linode
Deploy Nodejs app on a cloud VM such as GCP, AWS, Azure, Linode
Grafana Dashboard – Oscar Awards
Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040.
Rising IT Cities and Their Impact on the Economy, Environment, Infrastructure, and City Life in Future
Internet Demand’s Evolution, Communication Impact, and 2035’s Alternative Pathways
Rise of Cybercrime and its Effect in upcoming Future
AI/ML and Predictive Modeling
Solution for Contact Centre Problems
How to Setup Custom Domain for Google App Engine Application?

In [13]:


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

def extract_and_save_article(row):
    try:
        response = requests.get(row['URL'], headers=headers)
        response.raise_for_status()  
        soup = BeautifulSoup(response.text, 'html.parser')

        article_title = soup.find('h1').get_text()
        article_text = '\n'.join([p.get_text() for p in soup.find_all('p')])

        os.makedirs('article_texts', exist_ok=True)

        with open(f'article_texts/{int(row["URL_ID"])}.txt', 'w', encoding='utf-8') as file:
            file.write(article_title + '\n\n')
            file.write(article_text)

        print(f'Saved {article_title} to {int(row["URL_ID"])}.txt')
    except Exception as e:
        print(f'Error processing URL_ID {int(row["URL_ID"])}: {str(e)}')


Artical.apply(extract_and_save_article, axis=1)


Saved Rise of telemedicine and its Impact on Livelihood by 2040 to 123.txt
Saved Rise of e-health and its impact on humans by the year 2030 to 321.txt
Saved Rise of e-health and its impact on humans by the year 2030 to 2345.txt
Saved Rise of telemedicine and its Impact on Livelihood by 2040 to 4321.txt
Saved Rise of telemedicine and its Impact on Livelihood by 2040 to 432.txt
Saved Rise of Chatbots and its impact on customer support by the year 2040 to 2893.txt
Saved Rise of e-health and its impact on humans by the year 2030 to 3355.txt
Saved How does marketing influence businesses and consumers? to 3817.txt
Saved How advertisement increase your market value? to 4279.txt
Saved Negative effects of marketing on society to 4741.txt
Saved How advertisement/marketing affects business. to 5202.txt
Saved Rising IT cities will impact the economy, environment, infrastructure, and city life by the year 2035 to 5664.txt
Saved Rise of OTT platform and its impact on entertainment industry by the ye

Saved Coronavirus: Impact on the Hospitality Industry to 50921.txt
Saved Coronavirus impact on energy markets to 51382.txt
Saved What are the key policies that will mitigate the impacts of COVID-19 on the world of work? to 51844.txt
Saved Marketing Drives Results With A Focus On Problems to 52306.txt
Saved Continued Demand for Sustainability to 52768.txt


0      None
1      None
2      None
3      None
4      None
       ... 
109    None
110    None
111    None
112    None
113    None
Length: 114, dtype: object

Text Analysis

In [9]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:

default_stop_words = set(stopwords.words("english"))

stop_word_files = [
    "StopWords_Auditor.txt",
    "StopWords_Currencies.txt",
    "StopWords_DatesandNumbers.txt",
    "StopWords_Generic.txt",
    "StopWords_GenericLong.txt",
    "StopWords_Geographic.txt",
    "StopWords_Names.txt"
]

additional_stop_words = set()

for file in stop_word_files:
    with open(f"StopWords/{file}", "r") as stop_word_file:
        additional_stop_words.update(stop_word_file.read().splitlines())

stop_words = default_stop_words.union(additional_stop_words)


In [ ]:
positive_words = set()
negative_words = set()

with open("MasterDictionary/positive-words.txt", "r") as file:
    positive_words = set(file.read().splitlines())

with open("MasterDictionary/negative-words.txt", "r") as file:
    negative_words = set(file.read().splitlines())

In [ ]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

def calculate_sentiment(text):
    words = word_tokenize(text)
    positive_score = sum(1 for word in words if word in positive_words and word not in stop_words)
    negative_score = sum(1 for word in words if word in negative_words and word not in stop_words)
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)
    return positive_score, negative_score, polarity_score, subjectivity_score

def calculate_readability(text):
    sentences = sent_tokenize(text)
    total_words = len(word_tokenize(text))
    avg_sentence_length = total_words / len(sentences)
    complex_words = [word for word in word_tokenize(text) if len(word) > 2]
    percentage_complex_words = (len(complex_words) / total_words) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    return avg_sentence_length, percentage_complex_words, fog_index

def calculate_other_variables(text):
    words = word_tokenize(text)
    avg_words_per_sentence = len(words) / len(sent_tokenize(text))
    complex_word_count = sum(1 for word in words if len(word) > 2)
    word_count = len(words)
    syllable_count = sum(len(re.findall(r'[aeiouAEIOU]+', word)) for word in words)
    personal_pronouns = len(re.findall(r'\b(?:I|we|my|ours|us)\b', text, re.IGNORECASE))
    avg_word_length = sum(len(word) for word in words) / len(words)
    return avg_words_per_sentence, complex_word_count, word_count, syllable_count, personal_pronouns, avg_word_length


In [13]:

with open("article_texts/123.txt", "r") as file:
    text = file.read()


cleaned_text = clean_text(text)

positive_score, negative_score, polarity_score, subjectivity_score = calculate_sentiment(cleaned_text)

avg_sentence_length, percentage_complex_words, fog_index = calculate_readability(cleaned_text)

avg_words_per_sentence, complex_word_count, word_count, syllable_count, personal_pronouns, avg_word_length = calculate_other_variables(cleaned_text)


In [14]:

variables = [
    "POSITIVE SCORE",
    "NEGATIVE SCORE",
    "POLARITY SCORE",
    "SUBJECTIVITY SCORE",
    "AVG SENTENCE LENGTH",
    "PERCENTAGE OF COMPLEX WORDS",
    "FOG INDEX",
    "AVG NUMBER OF WORDS PER SENTENCE",
    "COMPLEX WORD COUNT",
    "WORD COUNT",
    "SYLLABLE PER WORD",
    "PERSONAL PRONOUNS",
    "AVG WORD LENGTH",
]

values = [
    positive_score,
    negative_score,
    polarity_score,
    subjectivity_score,
    avg_sentence_length,
    percentage_complex_words,
    fog_index,
    avg_words_per_sentence,
    complex_word_count,
    word_count,
    syllable_count,
    personal_pronouns,
    avg_word_length,
]


data = {'Variable': variables, 'Value': values}
df = pd.DataFrame(data)

print(df)


                            Variable        Value
0                     POSITIVE SCORE    83.000000
1                     NEGATIVE SCORE    28.000000
2                     POLARITY SCORE     0.495495
3                 SUBJECTIVITY SCORE     0.059742
4                AVG SENTENCE LENGTH  1858.000000
5        PERCENTAGE OF COMPLEX WORDS    84.337998
6                          FOG INDEX   776.935199
7   AVG NUMBER OF WORDS PER SENTENCE  1858.000000
8                 COMPLEX WORD COUNT  1567.000000
9                         WORD COUNT  1858.000000
10                 SYLLABLE PER WORD  3718.000000
11                 PERSONAL PRONOUNS     4.000000
12                   AVG WORD LENGTH     5.595264


**ALL ARTICALS**

In [29]:
import os
import re
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

default_stop_words = set(stopwords.words("english"))

stop_word_files = [
    "StopWords_Auditor.txt",
    "StopWords_Currencies.txt",
    "StopWords_DatesandNumbers.txt",
    "StopWords_Generic.txt",
    "StopWords_GenericLong.txt",
    "StopWords_Geographic.txt",
    "StopWords_Names.txt"
]

additional_stop_words = set()

for file in stop_word_files:
    with open(f"StopWords/{file}", "r") as stop_word_file:
        additional_stop_words.update(stop_word_file.read().splitlines())

stop_words = default_stop_words.union(additional_stop_words)
positive_words = set()
negative_words = set()

with open("MasterDictionary/positive-words.txt", "r") as file:
    positive_words = set(file.read().splitlines())

with open("MasterDictionary/negative-words.txt", "r") as file:
    negative_words = set(file.read().splitlines())

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

def calculate_sentiment(text):
    words = word_tokenize(text)
    positive_score = sum(1 for word in words if word in positive_words and word not in stop_words)
    negative_score = sum(1 for word in words if word in negative_words and word not in stop_words)
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)
    return positive_score, negative_score, polarity_score, subjectivity_score

def calculate_readability(text):
    sentences = sent_tokenize(text)
    total_words = len(word_tokenize(text))
    avg_sentence_length = total_words / len(sentences)
    complex_words = [word for word in word_tokenize(text) if len(word) > 2]
    percentage_complex_words = (len(complex_words) / total_words) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    return avg_sentence_length, percentage_complex_words, fog_index

def calculate_other_variables(text):
    words = word_tokenize(text)
    avg_words_per_sentence = len(words) / len(sent_tokenize(text))
    complex_word_count = sum(1 for word in words if len(word) > 2)
    word_count = len(words)
    syllable_count = sum(len(re.findall(r'[aeiouAEIOU]+', word)) for word in words)
    personal_pronouns = len(re.findall(r'\b(?:I|we|my|ours|us)\b', text, re.IGNORECASE))
    avg_word_length = sum(len(word) for word in words) / len(words)
    return avg_words_per_sentence, complex_word_count, word_count, syllable_count, personal_pronouns, avg_word_length


article_folder = "article_texts"


results_df = pd.DataFrame(columns=["URL_ID", "Variable", "Value"])

for filename in os.listdir(article_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(article_folder, filename), "r", encoding="utf-8") as file:
            text = file.read()

        cleaned_text = clean_text(text)
        positive_score, negative_score, polarity_score, subjectivity_score = calculate_sentiment(cleaned_text)
        avg_sentence_length, percentage_complex_words, fog_index = calculate_readability(cleaned_text)
        avg_words_per_sentence, complex_word_count, word_count, syllable_count, personal_pronouns, avg_word_length = calculate_other_variables(cleaned_text)

        url_id = os.path.splitext(filename)[0]
       
        results_df = results_df.append({"URL_ID": url_id, "Variable": "POSITIVE SCORE", "Value": positive_score}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "NEGATIVE SCORE", "Value": negative_score}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "POLARITY SCORE", "Value": polarity_score}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "SUBJECTIVITY SCORE", "Value": subjectivity_score}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG SENTENCE LENGTH", "Value": avg_sentence_length}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "PERCENTAGE OF COMPLEX WORDS", "Value": percentage_complex_words}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "FOG INDEX", "Value": fog_index}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG NUMBER OF WORDS PER SENTENCE", "Value": avg_words_per_sentence}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "COMPLEX WORD COUNT", "Value": complex_word_count}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "WORD COUNT", "Value": word_count}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "SYLLABLE PER WORD", "Value": syllable_count}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "PERSONAL PRONOUNS", "Value": personal_pronouns}, ignore_index=True)
        results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG WORD LENGTH", "Value": avg_word_length}, ignore_index=True)



C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POSITIVE SCORE", "Value": positive_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "NEGATIVE SCORE", "Value": negative_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POLARITY SCORE", "Value": polarity_score}, ignore_index=True)
C:\Users\Dell\App

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "NEGATIVE SCORE", "Value": negative_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POLARITY SCORE", "Value": polarity_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "SUBJECTIVITY SCORE", "Value": subjectivity_score}, ignore_index=True)
C:\Users

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "SUBJECTIVITY SCORE", "Value": subjectivity_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG SENTENCE LENGTH", "Value": avg_sentence_length}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "PERCENTAGE OF COMPLEX WORDS", "Value": percentage_complex_wo

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "PERCENTAGE OF COMPLEX WORDS", "Value": percentage_complex_words}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "FOG INDEX", "Value": fog_index}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG NUMBER OF WORDS PER SENTENCE", "Value": avg_words_per_sentenc

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:109: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "SYLLABLE PER WORD", "Value": syllable_count}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "PERSONAL PRONOUNS", "Value": personal_pronouns}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG WORD LENGTH", "Value": avg_word_length}, ignore_index=True)
C:\Us

...

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "WORD COUNT", "Value": word_count}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:109: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "SYLLABLE PER WORD", "Value": syllable_count}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "PERSONAL PRONOUNS", "Value": personal_pronouns}, ignore_index=True)
C:\Users\Dell\A

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "NEGATIVE SCORE", "Value": negative_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POLARITY SCORE", "Value": polarity_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "SUBJECTIVITY SCORE", "Value": subjectivity_score}, ignore_index=True)
C:\Users

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POSITIVE SCORE", "Value": positive_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "NEGATIVE SCORE", "Value": negative_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POLARITY SCORE", "Value": polarity_score}, ignore_index=True)
C:\Users\Dell\App

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "AVG NUMBER OF WORDS PER SENTENCE", "Value": avg_words_per_sentence}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "COMPLEX WORD COUNT", "Value": complex_word_count}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "WORD COUNT", "Value": word_count}, ignore_in

C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POSITIVE SCORE", "Value": positive_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "NEGATIVE SCORE", "Value": negative_score}, ignore_index=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_11044\1784686674.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"URL_ID": url_id, "Variable": "POLARITY SCORE", "Value": polarity_score}, ignore_index=True)
C:\Users\Dell\App

In [38]:
results_df.head()

,URL_ID,Variable,Value
0,10282,POSITIVE SCORE,64
1,10282,NEGATIVE SCORE,28
2,10282,POLARITY SCORE,0.391304
3,10282,SUBJECTIVITY SCORE,0.048652
4,10282,AVG SENTENCE LENGTH,1891.0


In [39]:
results_df_pivot = results_df.pivot(index='URL_ID', columns='Variable', values='Value')

In [41]:
results_df_pivot.head()

Variable,AVG NUMBER OF WORDS PER SENTENCE,AVG SENTENCE LENGTH,AVG WORD LENGTH,COMPLEX WORD COUNT,FOG INDEX,NEGATIVE SCORE,PERCENTAGE OF COMPLEX WORDS,PERSONAL PRONOUNS,POLARITY SCORE,POSITIVE SCORE,SUBJECTIVITY SCORE,SYLLABLE PER WORD,WORD COUNT
URL_ID,,,,,,,,,,,,,
10282,1891.0,1891.0,5.123744,1528,788.721523,28,80.803808,18,0.391304,64,0.048652,3343,1891
10744,1425.0,1425.0,5.089825,1168,602.785965,28,81.964912,20,0.282051,50,0.054737,2482,1425
11206,927.0,927.0,5.177994,765,403.809709,16,82.524272,9,0.333333,32,0.05178,1668,927
12129,896.0,896.0,5.002232,744,391.614286,17,83.035714,2,0.423729,42,0.065848,1544,896
123,1855.0,1855.0,5.598383,1567,775.789757,28,84.474394,4,0.495495,83,0.059838,3718,1855


In [48]:
results_df_pivot.reset_index(inplace=True)

In [50]:
results_df_pivot.head()

Variable,URL_ID,AVG NUMBER OF WORDS PER SENTENCE,AVG SENTENCE LENGTH,AVG WORD LENGTH,COMPLEX WORD COUNT,FOG INDEX,NEGATIVE SCORE,PERCENTAGE OF COMPLEX WORDS,PERSONAL PRONOUNS,POLARITY SCORE,POSITIVE SCORE,SUBJECTIVITY SCORE,SYLLABLE PER WORD,WORD COUNT
0,10282,1891.0,1891.0,5.123744,1528,788.721523,28,80.803808,18,0.391304,64,0.048652,3343,1891
1,10744,1425.0,1425.0,5.089825,1168,602.785965,28,81.964912,20,0.282051,50,0.054737,2482,1425
2,11206,927.0,927.0,5.177994,765,403.809709,16,82.524272,9,0.333333,32,0.05178,1668,927
3,12129,896.0,896.0,5.002232,744,391.614286,17,83.035714,2,0.423729,42,0.065848,1544,896
4,123,1855.0,1855.0,5.598383,1567,775.789757,28,84.474394,4,0.495495,83,0.059838,3718,1855


In [78]:
results_df_pivot.to_csv("result.csv", index=False)

In [4]:
Result=pd.read_csv("result.csv")

In [5]:
out_put=pd.read_excel("Output Data Structure.xlsx")

In [9]:
out_put.head(50)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2893.8,https://insights.blackcoffer.com/rise-of-chatb...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3355.6,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3817.4,https://insights.blackcoffer.com/how-does-mark...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4279.2,https://insights.blackcoffer.com/how-advertise...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4741.0,https://insights.blackcoffer.com/negative-effe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
out_put.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            114 non-null    float64
 1   URL                               114 non-null    object 
 2   POSITIVE SCORE                    0 non-null      float64
 3   NEGATIVE SCORE                    0 non-null      float64
 4   POLARITY SCORE                    0 non-null      float64
 5   SUBJECTIVITY SCORE                0 non-null      float64
 6   AVG SENTENCE LENGTH               0 non-null      float64
 7   PERCENTAGE OF COMPLEX WORDS       0 non-null      float64
 8   FOG INDEX                         0 non-null      float64
 9   AVG NUMBER OF WORDS PER SENTENCE  0 non-null      float64
 10  COMPLEX WORD COUNT                0 non-null      float64
 11  WORD COUNT                        0 non-null      float64
 12  SYLLABLE

In [13]:
Result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            112 non-null    int64  
 1   AVG NUMBER OF WORDS PER SENTENCE  112 non-null    int64  
 2   AVG SENTENCE LENGTH               112 non-null    int64  
 3   AVG WORD LENGTH                   112 non-null    float64
 4   COMPLEX WORD COUNT                112 non-null    int64  
 5   FOG INDEX                         112 non-null    float64
 6   NEGATIVE SCORE                    112 non-null    int64  
 7   PERCENTAGE OF COMPLEX WORDS       112 non-null    float64
 8   PERSONAL PRONOUNS                 112 non-null    int64  
 9   POLARITY SCORE                    112 non-null    float64
 10  POSITIVE SCORE                    112 non-null    int64  
 11  SUBJECTIVITY SCORE                112 non-null    float64
 12  SYLLABLE

In [14]:
Result.head(50)

,URL_ID,AVG NUMBER OF WORDS PER SENTENCE,AVG SENTENCE LENGTH,AVG WORD LENGTH,COMPLEX WORD COUNT,FOG INDEX,NEGATIVE SCORE,PERCENTAGE OF COMPLEX WORDS,PERSONAL PRONOUNS,POLARITY SCORE,POSITIVE SCORE,SUBJECTIVITY SCORE,SYLLABLE PER WORD,WORD COUNT
0,123,1855,1855,5.598383,1567,775.789757,28,84.474394,4,0.495495,83,0.059838,3718,1855
1,321,803,803,5.470735,659,354.026899,17,82.067248,5,0.433333,43,0.074720,1542,803
2,432,1416,1416,5.524718,1196,600.185311,31,84.463277,9,0.114286,39,0.049435,2673,1416
3,2345,1271,1271,5.321794,1056,541.633674,31,83.084186,5,-0.087719,26,0.044847,2300,1271
4,2893,1289,1289,5.511249,1096,549.610861,16,85.027153,6,0.492063,47,0.048875,2458,1289
5,3355,1248,1248,5.558494,1025,532.052564,14,82.131410,3,0.450980,37,0.040865,2395,1248
6,3817,1883,1883,5.333510,1548,786.083696,10,82.209241,7,0.726027,63,0.038768,3464,1883
7,4279,712,712,5.165730,620,319.631461,4,87.078652,3,0.529412,13,0.023876,1232,712
8,4321,1416,1416,5.524718,1196,600.185311,31,84.463277,9,0.114286,39,0.049435,2673,1416
9,4741,1209,1209,5.592225,1014,517.148387,51,83.870968,8,-0.291139,28,0.065343,2361,1209


In [15]:
out_put['URL_ID'] = out_put['URL_ID'].astype(int)

In [16]:
out_put.head(50)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2893,https://insights.blackcoffer.com/rise-of-chatb...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3355,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3817,https://insights.blackcoffer.com/how-does-mark...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4279,https://insights.blackcoffer.com/how-advertise...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4741,https://insights.blackcoffer.com/negative-effe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
merged_Dataframe= Result.merge(out_put[['URL_ID', 'URL']], on='URL_ID', how='left')

In [19]:
merged_Dataframe.head()

,URL_ID,AVG NUMBER OF WORDS PER SENTENCE,AVG SENTENCE LENGTH,AVG WORD LENGTH,COMPLEX WORD COUNT,FOG INDEX,NEGATIVE SCORE,PERCENTAGE OF COMPLEX WORDS,PERSONAL PRONOUNS,POLARITY SCORE,POSITIVE SCORE,SUBJECTIVITY SCORE,SYLLABLE PER WORD,WORD COUNT,URL
0,123,1855,1855,5.598383,1567,775.789757,28,84.474394,4,0.495495,83,0.059838,3718,1855,https://insights.blackcoffer.com/rise-of-telem...
1,321,803,803,5.470735,659,354.026899,17,82.067248,5,0.433333,43,0.074720,1542,803,https://insights.blackcoffer.com/rise-of-e-hea...
2,432,1416,1416,5.524718,1196,600.185311,31,84.463277,9,0.114286,39,0.049435,2673,1416,https://insights.blackcoffer.com/rise-of-telem...
3,2345,1271,1271,5.321794,1056,541.633674,31,83.084186,5,-0.087719,26,0.044847,2300,1271,https://insights.blackcoffer.com/rise-of-e-hea...
4,2893,1289,1289,5.511249,1096,549.610861,16,85.027153,6,0.492063,47,0.048875,2458,1289,https://insights.blackcoffer.com/rise-of-chatb...


In [21]:
columns = ["URL_ID", "URL", "POSITIVE SCORE", "NEGATIVE SCORE", "POLARITY SCORE", "SUBJECTIVITY SCORE", "AVG SENTENCE LENGTH", "PERCENTAGE OF COMPLEX WORDS", "FOG INDEX", "AVG NUMBER OF WORDS PER SENTENCE", "COMPLEX WORD COUNT", "WORD COUNT", "SYLLABLE PER WORD", "PERSONAL PRONOUNS", "AVG WORD LENGTH"]
merged_Dataframe = merged_Dataframe[columns]

In [22]:
merged_Dataframe.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123,https://insights.blackcoffer.com/rise-of-telem...,83,28,0.495495,0.059838,1855,84.474394,775.789757,1855,1567,1855,3718,4,5.598383
1,321,https://insights.blackcoffer.com/rise-of-e-hea...,43,17,0.433333,0.074720,803,82.067248,354.026899,803,659,803,1542,5,5.470735
2,432,https://insights.blackcoffer.com/rise-of-telem...,39,31,0.114286,0.049435,1416,84.463277,600.185311,1416,1196,1416,2673,9,5.524718
3,2345,https://insights.blackcoffer.com/rise-of-e-hea...,26,31,-0.087719,0.044847,1271,83.084186,541.633674,1271,1056,1271,2300,5,5.321794
4,2893,https://insights.blackcoffer.com/rise-of-chatb...,47,16,0.492063,0.048875,1289,85.027153,549.610861,1289,1096,1289,2458,6,5.511249


In [23]:
merged_Dataframe.to_csv("Final_output.csv",index=False)